<a href="https://colab.research.google.com/github/dmbk/Anomaly-Detection-System/blob/master/VidasaExtended_Conv3DLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install keras-layer-normalization
!pip install imageio

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from PIL import Image
from tensorflow.keras import layers
import time

from IPython import display

from os.path import join
from os import listdir
from os.path import isfile, join, isdir

#import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2DTranspose, Conv3DTranspose, ConvLSTM2D, BatchNormalization, LayerNormalization, TimeDistributed, Conv2D, Conv3D, MaxPool3D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model

import matplotlib.pyplot as plt
import argparse
from os.path import dirname
import cv2
from google.colab.patches import cv2_imshow

TensorFlow 2.x selected.


In [0]:
class Config:
    def __init__(self, data_dir):
        self.DATASET_PATH = join(data_dir,"UCSDped1/Train")
        self.TEST_DIR = join(data_dir,"UCSDped1/Test/")
        self.BATCH_SIZE = 4
        self.EPOCHS = 3
        self.MODEL_PATH = join(data_dir,"model_conv3d_lstm.hdf5")

conf = Config(data_dir="/content/drive/My Drive/UCSD_Anomaly_Dataset.v1p2/") 
physical_devices = tf.config.list_physical_devices('GPU') 
try: 
  tf.config.experimental.set_memory_growth(physical_devices[0], True) 
except: 
  # Invalid device or cannot modify virtual devices once initialized. 
  pass

In [60]:
def get_model(reload_model=True):

    if reload_model == True and os.path.isfile(conf.MODEL_PATH):
        model=load_model(conf.MODEL_PATH,custom_objects={'LayerNormalization': LayerNormalization})
        return model
 
    seq = Sequential()
    seq.add(TimeDistributed(Conv3D(256, (3, 11, 11), strides=(1,4,4), padding="same"), batch_input_shape=(None, 10, 16, 256, 256, 1)))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv3D(128, (3, 5, 5), strides=(1,2,2), padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv3D(64, (3, 3, 3), strides=(1,2,2), padding="same")))
    seq.add(LayerNormalization())
    # # # # #
    seq.add(TimeDistributed(ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True)))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(ConvLSTM2D(32, (3, 3), padding="same", return_sequences=True)))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True)))
    seq.add(LayerNormalization())
    # # # # #
    seq.add(TimeDistributed(Conv3DTranspose(64, (3, 3, 3), strides=(1,2,2), padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv3DTranspose(128, (3, 5, 5), strides=(1,2,2), padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv3DTranspose(256, (3, 11, 11), strides=(1,4,4), padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv3D(1, (5, 11, 11), activation="sigmoid", padding="same")))
    print(seq.summary(line_length=150))
    return seq

def get_loss(true_clips, pred_clips):
    return tf.keras.losses.MSE(y_true=true_clips, y_pred=pred_clips)

def get_optimizer():
    return tf.optimizers.Adam(1e-4, decay=1e-5, epsilon=1e-6)

@tf.function
def train_step(clips, model, optimizer):    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        reconstructed_clips = model(clips, training=True)
        reconstruction_loss = get_loss(reconstructed_clips, clips)

    gradients_of_model = gen_tape.gradient(reconstruction_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients_of_model, model.trainable_variables))
   
def train(dataset, conf):
    model = get_model()
    optimizer = get_optimizeer()
    for epoch in range(conf.EPOCHS):
        start = time.time()
    
        for clips_batch in dataset:
            train_step(clips_batch, model, optimizer)

        display.clear_output(wait=True)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    model.save(conf.MODEL_PATH,save_format='h5')
  
    display.clear_output(wait=True)


get_model()


Model: "sequential_47"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                                       Output Shape                                                Param #                
time_distributed_493 (TimeDistributed)                             (None, 10, 16, 64, 64, 256)                                 93184                  
______________________________________________________________________________________________________________________________________________________
layer_normalization_422 (LayerNormalization)                       (None, 10, 16, 64, 64, 256)                                 512                    
______________________________________________________________________________________________________________________________________________________
time_distributed_494 (TimeDistributed)                             (Non

In [0]:
def get_clips_by_stride(stride, frames_list, sequence_size):
    """ For data augmenting purposes.
    Parameters
    ----------
    stride : int
        The distance between two consecutive frames
    frames_list : list
        A list of sorted frames of shape 256 X 256
    sequence_size: int
        The size of the lstm sequence
    Returns
    -------
    list
        A list of clips , 32 frames each
    """
    clips = []
    sz = len(frames_list)
    clip = np.zeros(shape=(sequence_size, 256, 256, 1))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, sz, stride):
            clip[cnt, :, :, 0] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_size:
                clips.append(clip)
                cnt = 0
    return clips

def get_clips_list(seq_size):
    """
    seq_size :int 
        The sequence size of individual clip
    Returns
    -------
    list
        A list of training sequences of shape (NUMBER_OF_SEQUENCES,SINGLE_SEQUENCE_SIZE,FRAME_WIDTH,FRAME_HEIGHT,1)
    """
    clips = []
    # loop over the training folders (Train000,Train001,..)
    for f in sorted(listdir(conf.DATASET_PATH)):
        directory_path = join(conf.DATASET_PATH, f)
        if isdir(directory_path):
            all_frames = []
            # loop over all the images in the folder (0.tif,1.tif,..,199.tif)
            for c in sorted(listdir(directory_path)):
                img_path = join(directory_path, c)
                if str(img_path)[-3:] == "tif":
                    img = Image.open(img_path).resize((256, 256))

                    img = np.array(img, dtype=np.float32) / 256.0
                    all_frames.append(img)
            # get the 32-frames sequences from the list of images after applying data augmentation
            for stride in range(1, 3):
                clips.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=seq_size))
    return clips

def get_training_set(clip_seq_size):
    """
    clip_seq_size : int
        size of a single clips group
    Returns
    -------
    list
        creates list of lists of frames 
    """
    clips_array = []
    clips = get_clips_list(64)
    tmp_clip = []
    count = 0
    for clip in clips:
        tmp_clip.append(clip)
        count = count +1
        if count == clip_seq_size:
            clips_array.append(tmp_clip)
            count = 0


In [0]:
def get_single_test(single_test_path):
    sz = 200
    test = np.zeros(shape=(sz, 256, 256, 1))
    cnt = 0
    for f in sorted(listdir(single_test_path)):
        if str(join(single_test_path, f))[-3:] == "tif":
            img = Image.open(join(single_test_path, f)).resize((256, 256))
            #cv2_imshow(np.array(img,dtype=np.float32))
            #cv2.waitKey(0)
            img = np.array(img, dtype=np.float32) / 256.0
            test[cnt, :, :, 0] = img
            cnt = cnt + 1
    return test

def test_given_case(test_case_dir):
    test = get_single_test(join(conf.TEST_DIR,test_case_dir))
    print(test.shape)
   
    reconstructed_images = evaluate(test)
    test_discriminator(reconstructed_images)

In [0]:
def evaluate(test_case_dir):
    model = get_model(True)
    print("Model Loaded")
    test = get_single_test(join(conf.TEST_DIR,test_case_dir))
    print("Test case loaded")
    sz = test.shape[0] - 64
    sequences = np.zeros((sz, 64, 256, 256, 1))
    # apply the sliding window technique to get the sequences
    for i in range(0, sz):
        clip = np.zeros((64, 256, 256, 1))
        for j in range(0, 64):
            clip[j] = test[i + j, :, :, :]
        sequences[i] = clip

    sz_10x = sequences.shape[0] - 10
    sequences_10x = np.zeros((sz_10x, 10, 64, 256, 256, 1))
    count = 0
    for k in range(0, sz_10x):
        clip_10x = np.zeros((10, 64, 256, 256, 1))
        for l in range(0, 10):
            clip_10x[l] = sequences[k + l, :, :, :, :] 
        sequences_10x[k] = clip_10x

    # get the reconstruction cost of all the sequences
    reconstructed_sequences_10x = model.predict(sequences_10x,batch_size=4)
    #for j in range(0, sz):
    #    clip_10 = reconstructed_sequences[j]
    #    img_prev = np.reshape(clip_10[0],(256,256))*256
    #    for k in range(1,2):
    #        img = np.reshape(clip_10[k],(256,256))*256
            #optical_flow(img, img_prev)

    #        img_prev = img
            #print(img)
            #print("end of array\n")
            #cv2_imshow(img)
            #v2.waitKey(0)

    sr_cost_arr = []
    for idx in range(0, sz_10x):
        sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(sequences[idx][i],reconstructed_sequences[idx][i])) for i in range(0,sz)])
        sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / np.max(sequences_reconstruction_cost)
        sr = 1.0 - sa
        print(sr.shape())
        sr_cost_arr.append(sr)

    plt.plot(sr)
    plt.ylabel('regularity score Sr(t)')
    plt.xlabel('frame t')
    plt.show()
    sr = sigmoid(sr)
    # plot the regularity scores
    plt.plot(sr)
    plt.ylabel('regularity score Sr(t)')
    plt.xlabel('frame t')
    plt.show()